In [1]:
#the purpose of this script is to take a template adoc file and turns it into an adoc file ready to be converted into HTML.
#the template file includes main sections like Methods Arguments etc in double handle bars {{}}.  the script will replace 
#these and only these with appropriate asciidoc syntax

In [1]:
import sys
import re

In [2]:
#create a string to quickly test out functions
t = """hello {{method Jeremy}} there {{method close}}
{{intro 'Introduction and Overview'}} {{class fastai.transforms}} 
{{arguments TF}} {{notInFASTAI something}}  {{class Transform,tfm_y=TfmType.NO}}"""

In [3]:
#input string at top of ADOC to set style
'''
:icons: font
[%hardbreaks]

'''

'\n:icons: font\n[%hardbreaks]\n\n'

In [4]:
#list of functions used to substitute what's inside handle bars into ASCIIDOC syntax

In [5]:
def do_methods(ps):
    '''takes '{{methods}}' and returns '==Methods' '''
    return str("== Methods")

In [6]:
def do_method(ps):
    '''
    takes '{{method set_state, set_state_y=set_state.ARG}}' and returns 
    '::set_state  [.small]#(set_state_y=set_state.ARG)#'  
    '''
    codes = ([x.strip().replace("'","") for x in (str(ps)[1:-1]).split(',')])
    print(codes)
    return (str(codes[0])+"::" + " [.small]#set_state_y=set_state.ARG#")


In [7]:
def sub_arg(ps): 
    '''takes '{{Arguments}}'
and outputs
'== Arguments'
'''
    return '=='+str(ps)[2:-2]

In [250]:
def sub_args(ps): 
    '''
    takes 
    '{{arg tfm_y,TfmType ,TfmType.NO}]}' 
    and outputs
    'tfm_y: TfmType = {TfmType.NO}::'
    '''
    print("ps")
    print(ps)
    codes = ([x.strip() for x in (str(ps)[1:-1]).split(sep = ',')])
    print("codes")
    print(codes)
    
    
    
    dflt_value = " = {}".format(codes[2]) if len(codes) == 3 and len(codes[2]) > 0 else "" 
    print(dflt_value)
    return( "{0}: {1}{2}::".format(codes[0],codes[1],dflt_value) )



In [251]:
>>> import re
>>> string = "  blah, lots  ,  of ,  spaces, here "
>>> pattern = re.compile("^\s+|\s*,\s*|\s+$")
>>> print([x for x in pattern.split(string) if x])


['blah', 'lots', 'of', 'spaces', 'here']


In [252]:
def sub_class(ps): 
    ''' 
    takes '{{class Transform,tfm_y=TfmType.NO}}'
    and outputs
    '[[Transform]] 
     == Class Transform [.small]#(tfm_y=TfmType.NO)#'
 
    '''
    
    
    nameOfClass, arguments = ([x.strip() for x in (str(ps)[1:-1]).split(',')])
    return '[[{0}]]'.format(nameOfClass.strip()) + '\n' + '\n' + '==' + ' Class ' + nameOfClass + " [.small]#{0}#".format(arguments) + '\n'
        

In [253]:
#takes '{{xref TfmType http://link.org}}' and returns 'https://asciidoctor.org[TfmType]
def sub_xref(ps):
    return str(ps)

In [254]:
#takes '{{video https://www.youtube.com/watch?v=Th_ckFbc6bI}}' and returns
# for youtube   'video::Th_ckFbc6bI[youtube, width=300, start=60, end=140]'
# or for vimeo 'video::Th_ckFbc6bI[vimeo, width=300, start=60, end=140]'
# or "fastai only accepts videos from youtube and vimeo"

def sub_video(ps):
    return(ps)

In [255]:
#dictionary of main documentation sections (eg Methods, Arguments) and the function that replaces the syntax
fn_lu = { 
        'class': sub_class
        ,'method': do_method
        ,'methods': do_methods
        ,'arguments': sub_arg
        ,'args': sub_args
        ,'xref': sub_xref
        ,'video': sub_video }


In [282]:
def do_tmpl(s):
    '''
    parse template file into entries in handlebars {{}},
    call the respective function from fn_lu based on first handle bar entry...
    '''
    print("S")
    print(type(s))
    inner = s.group(1)
    print("inner", inner)
    fn_name,*params = inner.split(' ', 1)
    if fn_name not in fn_lu.keys():
        print("'{}'is not in FASTAI doc style guide".format(fn_name))
        return
    else:
        fn = fn_lu[fn_name]
        return fn(params)

In [283]:
#test string for class only
clsTstStr = """= fastai.transforms

== Transforms

{{class Transform,tfm_y=TfmType.NO}}"""



In [284]:
MthdTstsStr = '''
{{methods}}

{{method set_state, set_state_y=set_state.ARG}}
'''


In [285]:
MthdTstsStr = '''
{{methods}}

{{method set_state, set_state_y=set_state.ARG}}
'''


testSub = re.sub(r"{{(.*?)}}", do_tmpl, MthdTstsStr)
print(testSub)

S
<class '_sre.SRE_Match'>
inner methods
S
<class '_sre.SRE_Match'>
inner method set_state, set_state_y=set_state.ARG
['set_state', 'set_state_y=set_state.ARG']

== Methods

set_state:: [.small]#set_state_y=set_state.ARG#



In [286]:
# takes '{{class Transform,tfm_y=TfmType.NO}}' 

#and outputs 

#'''[[Transform]] 
#== Class Transform [.small]#(tfm_y=TfmType.NO)#'
#'''



testSub = re.sub(r"{{(.*?)}}", do_tmpl, clsTstStr)
print(testSub)

S
<class '_sre.SRE_Match'>
inner class Transform,tfm_y=TfmType.NO
= fastai.transforms

== Transforms

[['Transform]]

== Class 'Transform [.small]#tfm_y=TfmType.NO'#



In [287]:
#save file to adoc
with open("test.adoc", "w") as test:
    print(testSub, file=test)

In [296]:
argsTstStr = "{{args tfm_y,TfmType,   TfmTyp.NO}}"

In [297]:
#remove elements in handle bars and pass into do_tmpl.   tests on test string 'argsTstStr' first
#takes 
'''{{arg tfm_y,TfmType,TfmType.NO}}'''

#and outputs

'''tfm_y (type TfmType, default TfmType.NO)::'''


testSub = re.sub(r"{{(.*?)}}", do_tmpl, argsTstStr)
print(testSub)

S
<class '_sre.SRE_Match'>
inner args tfm_y,TfmType,   TfmTyp.NO
ps
['tfm_y,TfmType,   TfmTyp.NO']
codes
["'tfm_y", 'TfmType', "TfmTyp.NO'"]
 = TfmTyp.NO'
'tfm_y: TfmType = TfmTyp.NO'::


In [249]:
ls

abbr.md                     JG_transforms-tmpl.adoc  testing.adoc
anatomy.adoc                md_expander.py*          testTransform.adoc
dataloader.adoc             module-decisions.md      testTransform.html
expand_adoc_templ.ipynb*    README.md                transforms.adoc
expand_adoc_templ-JG.ipynb  style.md                 transforms.html
gen_ascii_docs.py           templates.py
__init__.py                 test.adoc


In [162]:
#open template file
tmpl = "JG_transforms-tmpl.adoc"
f = open(tmpl, "r")
contents = f.read()


In [163]:
print(contents)

= fastai.transforms

== Introduction and overview

The fastai transforms pipeline for images is designed to convert your independent and dependent variables into a form ready to be batched by your DataLoader and passed to your model. It is most commonly used like this:


```
...example...
```

The most common types of transforms are predefined in ...

The most likely customizations you might need to do are ...

You can create custom transform pipelines using an approach like: ...

If you want to create a custom transform, you will need to : ...

{{class Transform,tfm_y=TfmType.NO}}

.Abstract parent for all transforms.

Override do_transform to implement transformation of a single object.

{{Arguments}}

{{arg tfm_y,TfmType,TfmType.NO}}
Type of transform. For details, see {{xref TfmType}}.

{{methods}}

{{method set_state, set_state_arg}}

A transform may include a random component. If it does, it will often need to transform `y` using the same random values as `x` (e.g. a horizontal f

In [47]:
#do same on actual template file
rplcHndlbar = re.sub(r"{{(.*?)}}", do_tmpl, contents)

'Arguments'is not in FASTAI doc style guide
'arg'is not in FASTAI doc style guide
['set_state', '', 'set_state_arg']


In [48]:
print(rplcHndlbar)

= fastai.transforms

== Introduction and overview

The fastai transforms pipeline for images is designed to convert your independent and dependent variables into a form ready to be batched by your DataLoader and passed to your model. It is most commonly used like this:


```
...example...
```

The most common types of transforms are predefined in ...

The most likely customizations you might need to do are ...

You can create custom transform pipelines using an approach like: ...

If you want to create a custom transform, you will need to : ...

[[Transform]]

== Class Transform [.small]#tfm_y=TfmType.NO#


.Abstract parent for all transforms.

Override do_transform to implement transformation of a single object.




Type of transform. For details, see ['TfmType'].

== Methods

set_state:: [.small]#set_state_y=set_state.ARG#

A transform may include a random component. If it does, it will often need to transform `y` using the same random values as `x` (e.g. a horizontal flip in segment

In [49]:
#save file to adoc
with open("testTransform.adoc", "w") as finalAdoc:
    print(rplcHndlbar, file=finalAdoc)

In [50]:
!asciidoctor testTransform.adoc

In [51]:
#convert to HTML.  note you must have asciidoctor installed, which requires $pip install ruby then $ gem install asciidoctor
#!asciidoctor EDIT2_tmpl_transforms-tmpl.adoc

In [52]:
#ls

In [53]:
#load HTML just built
f2 = open("testTransform.html", "r")
page = f2.read()

In [54]:
#load HTML just built
#f2 = open("EDIT2_tmpl_transforms-tmpl.html", "r")
#page = f2.read()

In [55]:
#test html. 
#Note that disaplying this way gets video embeddings to work wheras opening html 
#directly without serving as website does not

from IPython.core.display import display, HTML
display(HTML(page))